In [1]:
# https://github.com/shenweichen/DeepMatch
import pandas as pd
from deepctr.feature_column import SparseFeat, VarLenSparseFeat
from preprocess import gen_data_set,gen_model_input
from sklearn.preprocessing import LabelEncoder
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import Model

from deepmatch.models import *
from deepmatch.utils import sampledsoftmaxloss

In [2]:
# def main():
data = pd.read_csv("./movielens_sample.txt")
data

,user_id,movie_id,rating,timestamp,title,genres,gender,age,occupation,zip
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama,F,1,10,48067
1,1,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical,F,1,10,48067
2,1,914,3,978301968,My Fair Lady (1964),Musical|Romance,F,1,10,48067
3,1,3408,4,978300275,Erin Brockovich (2000),Drama,F,1,10,48067
4,1,2355,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy,F,1,10,48067
...,...,...,...,...,...,...,...,...,...,...
228,3,3671,5,978297419,Blazing Saddles (1974),Comedy|Western,M,25,15,55117
229,3,2871,4,978297539,Deliverance (1972),Adventure|Thriller,M,25,15,55117
230,3,2115,4,978297777,Indiana Jones and the Temple of Doom (1984),Action|Adventure,M,25,15,55117
231,3,1136,5,978298079,Monty Python and the Holy Grail (1974),Comedy,M,25,15,55117


In [3]:
data.describe()

,user_id,movie_id,rating,timestamp,age,occupation,zip
count,233.000000,233.000000,233.000000,2.330000e+02,233.000000,233.000000,233.000000
mean,1.991416,1733.935622,3.862661,9.783286e+08,36.703863,14.416309,61793.158798
std,0.669479,1010.422343,0.950511,1.207502e+05,22.986181,2.434148,9536.743437
min,1.000000,1.000000,1.000000,9.782970e+08,1.000000,10.000000,48067.000000
25%,2.000000,1029.000000,3.000000,9.782985e+08,25.000000,15.000000,55117.000000
50%,2.000000,1545.000000,4.000000,9.782993e+08,56.000000,16.000000,70072.000000
75%,2.000000,2490.000000,5.000000,9.783001e+08,56.000000,16.000000,70072.000000
max,3.000000,3893.000000,5.000000,9.788244e+08,56.000000,16.000000,70072.000000


In [4]:
features = ['user_id', 'movie_id','gender', 'age', 'occupation', 'zip']
feature_max_idx = {}
for feature in features:
    lbe = LabelEncoder()
    data[feature] = lbe.fit_transform(data[feature]) + 1
    feature_max_idx[feature] = data[feature].max() + 1
data

,user_id,movie_id,rating,timestamp,title,genres,gender,age,occupation,zip
0,1,66,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama,1,1,1,1
1,1,39,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical,1,1,1,1
2,1,47,3,978301968,My Fair Lady (1964),Musical|Romance,1,1,1,1
3,1,191,4,978300275,Erin Brockovich (2000),Drama,1,1,1,1
4,1,147,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...
228,3,202,5,978297419,Blazing Saddles (1974),Comedy|Western,2,2,2,2
229,3,169,4,978297539,Deliverance (1972),Adventure|Thriller,2,2,2,2
230,3,135,4,978297777,Indiana Jones and the Temple of Doom (1984),Action|Adventure,2,2,2,2
231,3,64,5,978298079,Monty Python and the Holy Grail (1974),Comedy,2,2,2,2


In [5]:
data.describe()

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip
count,233.000000,233.000000,233.000000,2.330000e+02,233.000000,233.000000,233.000000,233.000000
mean,1.991416,103.300429,3.862661,9.783286e+08,1.772532,2.326180,2.326180,2.326180
std,0.669479,59.176131,0.950511,1.207502e+05,0.420100,0.823183,0.823183,0.823183
min,1.000000,1.000000,1.000000,9.782970e+08,1.000000,1.000000,1.000000,1.000000
25%,2.000000,54.000000,3.000000,9.782985e+08,2.000000,2.000000,2.000000,2.000000
50%,2.000000,102.000000,4.000000,9.782993e+08,2.000000,3.000000,3.000000,3.000000
75%,2.000000,153.000000,5.000000,9.783001e+08,2.000000,3.000000,3.000000,3.000000
max,3.000000,208.000000,5.000000,9.788244e+08,2.000000,3.000000,3.000000,3.000000


In [6]:
feature_max_idx

{'user_id': 4,
 'movie_id': 209,
 'gender': 3,
 'age': 4,
 'occupation': 4,
 'zip': 4}

In [11]:
user_profile = data[["user_id","gender","age","occupation","zip"]].drop_duplicates('user_id')
user_profile

,user_id,gender,age,occupation,zip
0,1,1,1,1,1
53,2,2,3,3,3
182,3,2,2,2,2


In [15]:
item_profile = data[["movie_id"]].drop_duplicates('movie_id')
item_profile

,movie_id
0,66
1,39
2,47
3,191
4,147
...,...
228,202
229,169
230,135
231,64


In [16]:
user_profile.set_index("user_id",inplace=True)
user_profile

,gender,age,occupation,zip
user_id,,,,
1,1,1,1,1
2,2,3,3,3
3,2,2,2,2


In [25]:
user_item_list = data.groupby("user_id")['movie_id'].apply(list)
user_item_list

user_id
1    [66, 39, 47, 191, 147, 68, 85, 166, 33, 48, 34...
2    [66, 180, 144, 127, 70, 132, 77, 89, 177, 100,...
3    [147, 68, 84, 126, 11, 183, 37, 71, 80, 168, 2...
Name: movie_id, dtype: object

In [26]:
negsample = 3
train_set, test_set = gen_data_set(data,negsample)

100%|██████████| 3/3 [00:00<00:00, 456.55it/s]

5 6


In [46]:
# 负样本：
# (1, [186], 168, 0, 1) 
# 正样本：
# (2,
#  [12, 177, 190, 167, 199, 194, 62, 46, 128, 161, 123, 113, 80, 205, 196, 201, 116, 58, 127, 49, 122, 125, 67, 89, 180, 78, 184, 6, 133, 176, 154, 59, 195, 25, 118, 32, 179, 70, 72, 121, 168, 175, 66, 14, 74, 173, 87, 160, 73, 71, 69],
#  143,
#  1,
#  51,
#  3)
# 参数说明：用户id、[历史观看视频id列表]、本次观看的视频、是否是正样本:0负样本，1正样本、历史列表长度、用户对本次观看电影的打分:1-5分；
train_set[0]

(1, [186], 168, 0, 1)